In [5]:
import nglview as nv
from Bio.PDB import *

import pickle
import pandas as pd

In [2]:
with open('pickles/cutoff_d.pickle', 'rb') as handle:
    cutoff_d = pickle.load(handle)

In [7]:
#collect lrrpredictor outputs into a dictionary
df = pd.read_csv('LRRPredictor_output/col-0-nlrome.txt', delimiter = '\t')
motifs_d = {}
for protid in cutoff_d:
    motifs = []
    foundProtein = False
    for ii in range(len(df)):
        if '.'.join(df['annotation'][ii].split('_')[1:]) == protid:
            foundProtein = True
        elif foundProtein:            
            break
        if foundProtein:
            motifs.append(int(df['start'][ii]))
    motifs_d[protid] = motifs

In [9]:
protid = 'AT3G44630.3'
pdb_parser = PDBParser()
structure = pdb_parser.get_structure(protid, f"pdb/{protid}.pdb")

# chain = list(list(structure.get_models())[0].get_chains())[0]

a, b = cutoff_d[protid]

scheme = []
prev = 0
nonmotifcolor = 'green'
motifcolor = 'red'
for motif in [pos for pos in motifs_d[protid] if pos <b]:
    scheme += [[nonmotifcolor, f"{prev}-{motif-1}"], [motifcolor, f"{motif}-{motif}"]]
    prev = motif+1
if motif < b:
    scheme.append([nonmotifcolor, f"{prev}-{b}"])

nv.color.ColormakerRegistry.add_selection_scheme(
    "my_custom_scheme", scheme
)

view = nv.show_biopython(structure, default_representation=False)
view.add_cartoon(color="my_custom_scheme")
view.center()
view

NGLWidget()